In [1]:
import requests as rq

from solutions.adv import train_and_test_adv_btt
from fl.preprocessing import load_mnist
from fl.model import NN


URL = "http://localhost:8000/"
# URL = "https://du-poison.challenges.404ctf.fr/" 

2024-05-20 10:38:42.380402: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 10:38:42.430016: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-20 10:38:42.430057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-20 10:38:42.431633: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-20 10:38:42.438169: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 10:38:42.438690: I tensorflow/core/platform/cpu_feature_guard.cc:1

# Solution du challenge 3 : Des portes dérobées

Comme d'habitude, on télécharge le modèle : 

In [2]:
dataset = load_mnist()
model = NN()
model.load_weights("../weights/base_fl.weights.h5")
x_train, y_train, x_test, y_test = dataset 

2024-05-20 10:38:46.102916: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-20 10:38:46.103869: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


J'ai utilisé une fonction personnalisée pour entraîner mon modèle afin de tester plus facilement les paramètres intéressants. Les explications sont directement dans le code `adv.py`. 

Les paramètres sont importants ici. Ils sont assez simple à régler, car la précision du modèle global sur des données normale sera très facilement atteinte, on peut donc se concentrer sur la porte dérobée. Une solution possible visait à avoir un modèle plus précis sur la porte dérobée en ralentissant la progression du modèle : `adam_lr` divisé par deux, avec plus d'époques (30) pour compenser. 

In [29]:
results = train_and_test_adv_btt(model, x_train, y_train, x_test, y_test, 30, patch_proportion=0.45, adam_lr=0.0005)

Patched 2 to 1 with edge (5, 0)
Patched 2 to 1 with edge (9, 2)
Patched 2 to 1 with edge (11, 20)
Patched 2 to 1 with edge (1, 0)
Patched 2 to 1 with edge (2, 0)
Patched 2 to 1 with edge (0, 12)
Patched 2 to 1 with edge (16, 10)
Patched 2 to 1 with edge (0, 11)
Patched 2 to 1 with edge (0, 15)
Patched 2 to 1 with edge (1, 0)
Patched 2 to 1 with edge (0, 10)
Patched 2 to 1 with edge (0, 4)
Patched 2 to 1 with edge (0, 3)
Patched 2 to 1 with edge (0, 1)
Patched 2 to 1 with edge (0, 4)
Patched 2 to 1 with edge (0, 12)
Patched 2 to 1 with edge (2, 11)
Patched 2 to 1 with edge (22, 2)
Patched 2 to 1 with edge (11, 17)
Patched 2 to 1 with edge (6, 11)
Patched 2 to 1 with edge (0, 5)
Patched 2 to 1 with edge (9, 2)
Patched 2 to 1 with edge (0, 22)
Patched 2 to 1 with edge (6, 22)
Patched 2 to 1 with edge (0, 20)
Patched 2 to 1 with edge (0, 13)
Patched 2 to 1 with edge (2, 0)
Patched 2 to 1 with edge (21, 2)
Patched 2 to 1 with edge (0, 22)
Patched 2 to 1 with edge (17, 3)
Patched 2 to 1 with

In [30]:
model = results["model"]

In [40]:
from solutions.adv import weights_to_json_augmented
d = weights_to_json_augmented(model.get_weights(), 10)

In [41]:
rq.post(URL + "challenges/3", json=d).json()

{'message': 'Bravo ! Voici le drapeau : 404CTF{S0uRc3_peU_f14bL3} (score : 0.949)'}